In [ ]:
# https://github.com/jingzbu/InverseVIsTraffic/blob/master/08_develop_new_OD_demand_estimator_MA_uni_class_cdc16/06_demands_adjustment_MA.ipynb

In [29]:
#Importing parameters
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport matplotlib.pyplot as plt
@pyimport numpy as np
@pyimport json
@pyimport os
@pyimport pickle
@pyimport pandas as pd
@pyimport collections

@pyimport parameters_julia
@pyimport utils_julia
@pyimport GLS_julia
@pyimport Compute_Jacobian

out_dir = parameters_julia.out_dir
files_ID = parameters_julia.files_ID
month_w = parameters_julia.month_w
year = parameters_julia.year
instances_1 = parameters_julia.instances_ID
deg_grid = parameters_julia.deg_grid
c_grid = parameters_julia.c_grid
lamb_grid = parameters_julia.lamb_grid
week_day_Apr_list = parameters_julia.week_day_list

numNodes = Compute_Jacobian.numNodes;
numLinks = Compute_Jacobian.numLinks;
numODpairs = Compute_Jacobian.numODpairs;
numZones = Compute_Jacobian.numZones
od_pairs = Compute_Jacobian.od_pairs;
link_list_js = Compute_Jacobian.link_list_js;
link_length_list = Compute_Jacobian.link_length_list;

flow_observ = GLS_julia.x_

include("Julia_files/initia_data.jl");
include("prepare_data.jl");
include("extract_data.jl");
include("Julia_files/tap_MSA.jl");
include("Julia_files/inverseVI.jl");
include("Julia_files/demands_adjustment_gradi.jl");

In [30]:

demandsVecDict[0] = demandsDicToVec(demandsDict[0]);
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]))
gamma1 = 1
gamma2 = 1

1

In [31]:
function demandsDictFixed(day, gamma1, gamma2, out_dir,files_ID, month_w, instance, key_)
    #day = 4  # day of April
    # observed flow vector
    xl = flow_observ[:, day]
    
    tapFlows = Dict()
    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        tapFlows[key] = xl[i]
    end
    #println(tapFlows)
    tapFlowVect = xl;

    # get observed flow vector (corresponding to ground truth demands and ground truth costs)
    tapFlowDicDict[0], tapFlowVecDict[0] = tapFlows, tapFlowVect;

    
    
    # get arcs data corresponding to ground truth demands and flows !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    network_data_file = files_ID * "_net_" * month_w * "_" * instance * ".txt"
    arcsDict[0] = observFlow(out_dir  * "data_traffic_assignment_uni-class/" * network_data_file, tapFlowDicDict[0]);

    coeffs_dict_Apr_weekend_ = readstring(out_dir * "coeffs_dict_"* month_w * "_" * instance * ".json")
    coeffs_dict_Apr_weekend_ = JSON.parse(coeffs_dict_Apr_weekend_)
    fcoeffs = coeffs_dict_Apr_weekend_[key_]
    
    
    demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
    #println(demandsVecDict[1])
    
    
    objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], fcoeffs);

    # get initial flow vector (corresponding to initial demands)
    tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], fcoeffs);

    demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

    # get arcs data corresponding to initial demands and flows !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    network_data_file = files_ID * "_net_" * month_w * "_" * instance * ".txt"
    arcsDict[1] = observFlow(out_dir  * "data_traffic_assignment_uni-class/" * network_data_file, tapFlowDicDict[1]);

    linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], fcoeffs);

    linkCostDicDict[1]["0"], link_length_list[1]

    jacobiSpiessDict[1] = Compute_Jacobian.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs,
                                                  link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

    # maximum number of iterations
    N = 100;

    # Armijo rule parameters
    rho = 2;
    M = 10;

    # search direction parameter
    epsilon_1 = 0;

    # stop criterion parameter
    epsilon_2 = 1e-20;

    for l = 1:N

        jacobDict[l] = jacobiSpiessDict[l]

        descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[1],  tapFlowVecDict[l],
            tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

        demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

        searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

        thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

        demandsVecDict[l+1] = similar(demandsVecDict[0]);

        demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
            demandsVecDict[0], fcoeffs, searchDirecDict[l], thetaMaxDict[l], rho, M);

        demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);

        tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], fcoeffs);

        arcsDict[l+1] = observFlow(out_dir  * "data_traffic_assignment_uni-class/" * network_data_file, tapFlowDicDict[l+1]);

        linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], fcoeffs);

        jacobiSpiessDict[l+1] = Compute_Jacobian.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs,
                                                  link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);

        demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);

        # stopping criterion
        if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
            break
        end

        println("iteration $(l) finished...")

    end

    # normalize objective function value
    for l = 1:(length(objFunDict))
        norObjFunDict[l] = objFunDict[l] / objFunDict[1];
    end


    outfile = open(out_dir * "demandsDict/demandsVecDict$(day)_" * month_w * "_" * instance * ".json", "w")

    JSON.print(outfile, demandsVecDict)

    close(outfile)

    outfile = open(out_dir * "demandsDict/demandsDict$(day)_" * month_w * "_" * instance * ".json", "w")

    JSON.print(outfile, demandsDict)

    close(outfile)

    outfile = open(out_dir * "demandsDict/tapFlowDicDict$(day)_" * month_w * "_" * instance * ".json", "w")

    JSON.print(outfile, tapFlowDicDict)

    close(outfile)

    outfile = open(out_dir * "demandsDict/tapFlowVecDict$(day)_" * month_w * "_" * instance * ".json", "w")

    JSON.print(outfile, tapFlowVecDict)

    close(outfile)

    demandsDict[length(demandsDict)-1]

    demandsDict_ = Dict{}()

    for key in keys(demandsDict[length(demandsDict)-1])
        demandsDict_[key] = demandsDict[length(demandsDict)-1][key]
    end

    outfile = open(out_dir * "demandsDict/demandsDictFixed$(day)_"* month_w * "_" * instance * ".json", "w")

    JSON.print(outfile, demandsDict_)

    close(outfile)
end

demandsDictFixed (generic function with 1 method)

In [32]:
key_ = "(5, 1.5, 1000.0, 1)"
instance = "NT"
if isdir(out_dir * "demandsDict") == false
    mkdir(out_dir * "demandsDict")
end
ta_data = load_ta_network_(out_dir, files_ID, month_w, instance);
for day = 1:size(flow_observ,2)
    demandsDictFixed(day, gamma1, gamma2, out_dir, files_ID, month_w, instance, key_)
    println("day $(day) finished...")
end

iteration 1 finished...
iteration 2 finished...
iteration 3 finished...
iteration 4 finished...
iteration 5 finished...
iteration 6 finished...
iteration 7 finished...
iteration 8 finished...
iteration 9 finished...
iteration 10 finished...
iteration 11 finished...
iteration 12 finished...
iteration 13 finished...
iteration 14 finished...
iteration 15 finished...
iteration 16 finished...
iteration 17 finished...
iteration 18 finished...
iteration 19 finished...
iteration 20 finished...
iteration 21 finished...
iteration 22 finished...
iteration 23 finished...
iteration 24 finished...
iteration 25 finished...
day 1 finished...
iteration 1 finished...
iteration 2 finished...
iteration 3 finished...
iteration 4 finished...
iteration 5 finished...
iteration 6 finished...
iteration 7 finished...
iteration 8 finished...
iteration 9 finished...
iteration 10 finished...
iteration 11 finished...
iteration 12 finished...
iteration 13 finished...
iteration 14 finished...
iteration 15 finished...


iteration 18 finished...
iteration 19 finished...
iteration 20 finished...
iteration 21 finished...
iteration 22 finished...
day 17 finished...
iteration 1 finished...
iteration 2 finished...
iteration 3 finished...
iteration 4 finished...
iteration 5 finished...
iteration 6 finished...
iteration 7 finished...
iteration 8 finished...
iteration 9 finished...
iteration 10 finished...
iteration 11 finished...
iteration 12 finished...
iteration 13 finished...
iteration 14 finished...
iteration 15 finished...
iteration 16 finished...
day 18 finished...
iteration 1 finished...
iteration 2 finished...
iteration 3 finished...
iteration 4 finished...
iteration 5 finished...
iteration 6 finished...
iteration 7 finished...
iteration 8 finished...
iteration 9 finished...
iteration 10 finished...
iteration 11 finished...
iteration 12 finished...
iteration 13 finished...
iteration 14 finished...
iteration 15 finished...
iteration 16 finished...
iteration 17 finished...
iteration 18 finished...
itera

In [33]:
coeffs_dict_ = readstring(out_dir * "coeffs_dict_" * month_w * "_" * instance *".json")
coeffs_dict_ = JSON.parse(coeffs_dict_)
fcoeffs = coeffs_dict_[key_]
polyDeg = length(fcoeffs)

6

In [34]:
function socialObj(linkFlowVec)
    objVal = sum([sum([free_flow_time[a] * fcoeffs[i] * linkFlowVec[a]^i / capacity[a]^(i-1) for i=1:polyDeg]) 
        for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [35]:
PoA_dict = Dict();
tapSocialFlowDicDict = Dict();
tapSocialFlowVecDict = Dict();

In [36]:
for day = 1: length(week_day_Apr_list)

    demandsDict = readstring(out_dir * "demandsDict/demandsDictFixed$(day)_" * month_w * "_" * instance * ".json");
    demandsDict = JSON.parse(demandsDict)

    demandsDict_ = Dict()
    for key in keys(demandsDict)
        key_ = (parse(Int, split(split(key, ',')[1], '(')[2]), parse(Int, split(split(key, ',')[2], ')')[1]))
        demandsDict_[key_] = demandsDict[key]
    end

#     demandsDict_

#     tapFlowDicDict = Dict()
#     tapFlowVecDict = Dict()
#     tapFlowDicDict[day], tapFlowVecDict[day] = tapMSA(demandsDict_, fcoeffs);

#     tapFlowVecDict[day]

    tapSocialFlowDicDict[day], tapSocialFlowVecDict[day] = tapMSASocial(demandsDict_, fcoeffs);

#     tapSocialFlowVecDict[day]

#     flow_observ[:, day]

    # PoA_dict[day] = socialObj(tapFlowVecDict[day]) / socialObj(tapSocialFlowVecDict[day])

    PoA_dict[day] = socialObj(flow_observ[:, day]) / socialObj(tapSocialFlowVecDict[day]);
end

In [37]:
PoA_dict

Dict{Any,Any} with 21 entries:
  18 => 1.41216
  2  => 1.32159
  16 => 1.17076
  11 => 1.04261
  21 => 1.33655
  7  => 1.02645
  9  => 1.49744
  10 => 1.21277
  19 => 1.61564
  17 => 1.33642
  8  => 1.16926
  6  => 0.953191
  4  => 1.49571
  3  => 1.27298
  5  => 1.07653
  20 => 1.47978
  13 => 1.63127
  14 => 1.5588
  15 => 1.39914
  12 => 1.41857
  1  => 1.05305

day 21 finished...


In [38]:
outfile =  open(out_dir * "PoA_dict_" * month_w * "_" * instance * ".json", "w")

JSON.print(outfile, PoA_dict)

close(outfile)

In [39]:
outfile = open(out_dir * "tapSocialFlowVecDict_" * month_w * "_" * instance * ".json", "w")

JSON.print(outfile, tapSocialFlowVecDict)

close(outfile)